In [1]:
import scipy
from scipy import stats
import pandas as pd
import math

* V prvnim pripade se jedna o parovy test (pred/po -- parova data)
* V druhem pripade se jedna o dvouvyberovy t-test (dva vybery)

# RUCE

In [123]:
df = pd.read_csv("data-ruce.txt", delimiter=" ")

# two-sided test
# nevim jestli pocet zachvevu ocekavat mensi nebo vetsi
# zajima me, jestli se to proste lisi
stats.ttest_rel(df["ruka_pred"], df["ruka_po"])

Ttest_relResult(statistic=-3.1814457556571267, pvalue=0.00873863596242495)

* H0: same E(X)
* H1: different E(X)
* a = 0.01
* pvalue=0.008 < 0.01 ... **zamitame H0** - zamitame stejne stredni hodnoty

### pripominky k t-testum:
* 1/ **v prvni casti jde zrejme o parovy test (ne o "pravy")**
    *  ano byl to preklep
* 2/ **pouzil jednostranny test - rozmyslete jake duvody pro pouziti jednostranneho / oboustranneho testu**
   * po uvaze byc asi takel volil radeji jednostranny test:
   * zajima nas, jestli zubum skodime natolik, ze vydrzi mene ...
* 3/ **vypocet konfidencniho intervalu je zjevne chybny [t-test nezamita-> interval musi obsahovat hodnotu]**
  * Chyba spocivala v tom, ze jsem pro two-tailed pouzil hodnotu alpha a ne aplha/2
  * to me zmatlo proto, ze funkce `scipy.stats.ttest_ind()` dava uz rovnou p-value pro two-tailed test
  
  * zde jsem znovu cele projistotu prepocital:

# ZUBY

In [124]:
data = pd.read_csv("data-zuby.txt", delimiter=" ")
s_1, s_2 = data["zuby_pomlau"], data["zuby_sok"]
n_1, n_2 = s_1.size, s_2.size
# data.boxplot()

# alpha
a=0.05

# std err
se_p = math.sqrt(s_1.var() + (s_2.var)()) / math.sqrt(n_2) 

# mu 
mu = (s_1.mean()- s_2.mean())

# t stat
t_stat = mu / se_p

# df
df = n_1 -1 + n_2 - 1 # same variance 
df = se_p**4 / (1/(n_1 - 1) * (s_1.var()/n_1)**2 +  1/(n_1 - 1) * (s_2.var()/n_1)**2) # unkown variance


# p values for t_stat
pval_two_sided = stats.t.sf(abs(t_stat), df)*2  # Prob(abs(t)>t_stat)
pval_one_sided = stats.t.sf(abs(t_stat), df)    # Prob(abs(t)>t_stat) / 2


# t critic for df, alpha
t_crit_two_sided = stats.t.ppf(q=1-a/2, df=df) # a/2
t_crit_one_sided = stats.t.ppf(q=1-a, df=df)  

# iterval two sided
interval_L = mu - t_crit_two_sided*se_p
interval_R = mu + t_crit_two_sided*se_p

## TWO-SAMPLE TEST (TWO-TAILED)
### H0: mu(pomalu varene) == mu(rychle varene)
### H1: mu(pomalu varene) != mu(rychle varene)

In [125]:
# TEST BY VALUE OF STATISTICS
if  abs(t_stat) > t_crit_two_sided:
    print("|t_stat| >= t_crit : {} > {}".format(t_stat, t_crit_two_sided))
    print("\t REJECTING H0 in favor of H1")
    print("\t H1: mu(pomalu varene)  != mu(rychle varene)  ")
    print("")

else:
    print("|t_stat| < t_crit : {} <= {}".format(t_stat, t_crit_two_sided))
    print("\t Failed to reject H0")
    print("\t H0: mu(rychle varene) ==  mu(pomalu varene) ")
    print("")


# TEST BY P VALUE
if pval_two_sided < a:
    print("p_val < alpha : {} < {}".format(pval_two_sided, a))
    print("\t REJECTING H0 in favor of H1")
    print("\t H1: mu(pomalu varene)  > mu(rychle varene)")
    print("")

else:
    print("p_val >= alpha : {} >= {}".format(pval_two_sided, a))
    print("\t Failed to reject H0")
    print("\t H0: mu(rychle varene) ==  mu(pomalu varene) ")
    print("")

    
# INTERVAL
interval_L = mu - t_crit_two_sided*se_p
interval_R = mu + t_crit_two_sided*se_p
print("(1-{})% conf. interval ({},{})".format(a, interval_L, interval_R))

|t_stat| < t_crit : 2.019354722091951 <= 2.2042005991260605
	 Failed to reject H0
	 H0: mu(rychle varene) ==  mu(pomalu varene) 

p_val >= alpha : 0.06879844307365293 >= 0.05
	 Failed to reject H0
	 H0: mu(rychle varene) ==  mu(pomalu varene) 

(1-0.05)% conf. interval (-0.1064118798452327,2.4314118798452284)


## TWO-SAMPLE  (ONE-TAILED TEST)
### H0: mu(rychle varene) >=  mu(pomalu varene) 
### H1: mu(pomalu varene)  > mu(rychle varene)  

In [136]:
# TEST BY VALUE OF STATISTICS
if t_stat < t_crit_one_sided:
    print("t_stat < t_crit : {} < {}".format(t_stat, t_crit_one_sided))
    print("\t Failed to reject H0")
    print("\t H0: mu(rychle varene) >=  mu(pomalu varene) ")
    print("")
    
else:
    print("t_stat >= t_crit : {} >= {}".format(t_stat, t_crit_one_sided))
    print("\t REJECTING H0 in favor of H1")
    print("\t H1: mu(pomalu varene) > mu(rychle varene)  ")
    print("")

# TEST BY P VALUE
if pval_one_sided < a:
    print("p_val < alpha : {} < {}".format(pval_one_sided, a))
    print("\t REJECTING H0 in favor of H1")
    print("\t H1: mu(pomalu varene) > mu(rychle varene)  ")
    print("")

else:
    print("p_val >= alpha : {} >= {}".format(pval_one_sided, a))
    print("\t Failed to reject H0")
    print("\t H0: mu(rychle varene) >=  mu(pomalu varene) ")
    print("")

# INTERVAL
interval_L = mu - t_crit_one_sided*se_p
interval_R = mu + t_crit_one_sided*se_p
print("(1-{})% conf. interval ({:.4f},+inf)".format(a, interval_L))

t_stat >= t_crit : 2.019354722091951 >= 1.7978520481089464
	 REJECTING H0 in favor of H1
	 H1: mu(pomalu varene) > mu(rychle varene)  

p_val < alpha : 0.034399221536826466 < 0.05
	 REJECTING H0 in favor of H1
	 H1: mu(pomalu varene) > mu(rychle varene)  

(1-0.05)% conf. interval (0.1275,+inf)
